In [85]:
import numpy as np
import random, string

#def random_label(N = 2):
#    return ''.join(random.choices(string.ascii_uppercase + string.digits, k=N))

def random_label(bits = 64):
    return hex(random.getrandbits(bits))[2:]


In [86]:
n_bits = 2

t = []

label_to_random_label = {}
random_label_to_label = {}

circuit_per_bit = {}
for i in range(n_bits):
    circuit_per_bit[i] = []

for i in range(2**n_bits):
    for j in range(2**n_bits):
        format_string = '0{}b'.format(n_bits)
        
        a_in  = format(i, format_string)
        b_in  = format(j, format_string)
        c_out = format(min(i,j), format_string)
        
        t.append((a_in, b_in, c_out))
        for idx, bit in enumerate(c_out):
            if int(bit) != 1: continue
            
            a = ''
            b = ''
            
            for n in range(n_bits):
                a_i = a_in[n]
                b_i = b_in[n]
                
                label_a = ('-' if a_i == '0' else '') + ('a'+str(n))
                label_b = ('-' if b_i == '0' else '') + ('b'+str(n))
                
                while True:
                    if label_a in label_to_random_label.keys():
                        break
                    r_label_a = random_label()
                    if not r_label_a in random_label_to_label.keys():
                        random_label_to_label[r_label_a] = label_a
                        label_to_random_label[label_a] = r_label_a
                        break
                while True:
                    if label_b in label_to_random_label.keys():
                        break
                    r_label_b = random_label()
                    if not r_label_b in random_label_to_label.keys():
                        random_label_to_label[r_label_b] = label_b  
                        label_to_random_label[label_b] = random_label()
                        break
                
                a += str(a_i)
                b += str(b_i)
            
            circuit_per_bit[idx].append(a+b)
            
                
circuit_per_bit

{0: ['1010', '1011', '1110', '1111'],
 1: ['0101', '0110', '0111', '1001', '1101', '1111']}

In [87]:
random_label_to_label

{'a409bb31125e68e6': '-a0',
 '62398cc4c70778aa': '-b0',
 'a5b5f9a74a2c1fc6': 'a1',
 '457686f7b3ed37bc': 'b1',
 'd89caa899c35c6b6': 'b0',
 '36f7db245a72e2ec': '-b1',
 'c94b9c0902827d0c': 'a0',
 '331962e4e774b97a': '-a1'}

In [21]:
from KMap import Minterms
from utils import Term

str_terms = ['0110', '1000', '1001', '1010', '1011', '1100', '1101', '1110']
t_minterms = [Term(term) for term in str_terms]

minterms = Minterms(t_minterms)
minterms.simplify()
print(minterms.result)



[1*0*, 10**, *110]


In [48]:
from KMap import Minterms
from utils import Term

circuits = []

for key in result_per_bit.keys():
    terms = result_per_bit[key]
    t_minterms = [Term(term) for term in terms]

    minterms = Minterms(t_minterms)
    minterms.simplify()
    circuits.append(minterms.result)
    
circuits
    

[[1***1***],
 [1***01**, 01**1***, *1***1**],
 [11***01*, *1**001*, **1***1*, 001**1**, *01*11**, 1***0*1*, 0*1*1***],
 [111***01,
  11***0*1,
  1*1**001,
  **1*0001,
  1***0**1,
  *11*0*01,
  0001**1*,
  **01111*,
  ***1***1,
  00*1*1**,
  *0*111**,
  0**11***,
  0*01*11*,
  *1**00*1,
  *0011*1*]]

In [49]:
import random

garbled = True

for circuit in circuits:
    result = []
    for c in circuit:
        s = []
        for i, bit in enumerate(str(c)):
            if bit == '*': continue
            label = ('-' if bit == '0' else '') + ('a'+str(i%n_bits) if i < n_bits else 'b'+str(i%n_bits))
            s.append(label_to_random_label[label] if garbled else label)
        
        if garbled:
            random.shuffle(s)
        result.append('^'.join(s))
    if garbled:
        random.shuffle(result)
    print(' v '.join(result))
    

0x90aecfaaa36102c3^0x53cd87bfdd08fa16
0x449376f5c0ae61cd^0x4fc8bf4294a81c70 v 0x90aecfaaa36102c3^0x2b7f939881f2e084^0x4fc8bf4294a81c70 v 0x449376f5c0ae61cd^0x53cd87bfdd08fa16^0xfe7e1f8a2c1c0598
0x2b7f939881f2e084^0x90aecfaaa36102c3^0x53cd87bfdd08fa16 v 0xfe7e1f8a2c1c0598^0x53cd87bfdd08fa16^0x53cd87bfdd08fa16 v 0x53cd87bfdd08fa16^0x58eb8f2785c78d86^0x4fc8bf4294a81c70^0x53cd87bfdd08fa16 v 0x53cd87bfdd08fa16^0x2b7f939881f2e084^0x449376f5c0ae61cd^0xe86799ccb0a6a9cf v 0x449376f5c0ae61cd^0x90aecfaaa36102c3^0xe86799ccb0a6a9cf^0x53cd87bfdd08fa16 v 0x53cd87bfdd08fa16^0x53cd87bfdd08fa16 v 0x4fc8bf4294a81c70^0x53cd87bfdd08fa16^0xfe7e1f8a2c1c0598^0x58eb8f2785c78d86
0x90aecfaaa36102c3^0x2b7f939881f2e084^0x53cd87bfdd08fa16^0x4fc8bf4294a81c70^0xe86799ccb0a6a9cf v 0x2b7f939881f2e084^0x449376f5c0ae61cd^0x4fc8bf4294a81c70^0x2b7f939881f2e084^0x53cd87bfdd08fa16 v 0x2b7f939881f2e084^0x58eb8f2785c78d86^0x4fc8bf4294a81c70^0x53cd87bfdd08fa16^0x53cd87bfdd08fa16 v 0x4fc8bf4294a81c70^0x2b7f939881f2e084^0x90aecfa

In [93]:
from aes_cbc import AESCipher

v = [value for value in label_to_random_label.values()]
#v[0].decode('hex')
print(v[0])
bytes.fromhex(v[2]).decode('utf-16')

a409bb31125e68e6


'떥ꟹⱊ옟'